In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import os

ModuleNotFoundError: No module named 'transformers'

In [ ]:
model_path = ""
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
data_file = "LLM_as_a_judge/LLMEval2_benchmark.json"  # 修改为你的数据集路径

with open(data_file, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

print(f"共加载 {len(dataset)} 条数据")

In [ ]:
correct_predictions = 0
total_predictions = 0
evaluations = []

for entry in dataset:
    query = entry["query"]
    response1 = entry["response1"]
    response2 = entry["response2"]
    human_label = entry["human"]

    # 构造 prompt
    prompt = \
        f"""
        You are an expert language evaluator. Compare the following two responses to the given query and determine which one is better in terms of clarity, helpfulness, relevance, and fluency.

        Query: "{query}"
        Response 1: "{response1}"
        Response 2: "{response2}"
        
        Which response is better? Reply with "1" or "2" and briefly explain why.
        """

    # 拼接对话格式
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 编码输入
    inputs = tokenizer(text, return_tensors="pt")

    # 生成响应，开启返回 score
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        return_dict_in_generate=True,
        output_scores=True
    )

    # 解码输出内容
    response_ids = output.sequences[0][len(inputs.input_ids[0]):].tolist()
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

    # 计算信心度：softmax 后的最大概率均值
    scores = output.scores
    token_probs = torch.nn.functional.softmax(torch.cat(scores), dim=-1)
    max_probs = torch.max(token_probs, dim=-1).values
    confidence = max_probs.mean().item()

    # 判断模型输出的是选1还是选2
    if "1" in response_text:
        model_prediction = "1"
    elif "2" in response_text:
        model_prediction = "2"
    else:
        model_prediction = "unknown"

    # 对比标签，统计准确率
    if model_prediction == human_label:
        correct_predictions += 1
    total_predictions += 1

    # 保存结果
    evaluations.append({
        "query": query,
        "response1": response1,
        "response2": response2,
        "human": human_label,
        "model_output": response_text.strip(),
        "model_choice": model_prediction,
        "confidence": confidence
    })

In [ ]:
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0
print(f"🌟 模型评估准确率：{accuracy * 100:.2f}%")

In [ ]:
for i, eval_item in enumerate(evaluations[:5]):
    print(f"📝 示例 {i+1}")
    print("问题:", eval_item["query"])
    print("响应 1:", eval_item["response1"])
    print("响应 2:", eval_item["response2"])
    print("人类标签:", eval_item["human"])
    print("模型输出:", eval_item["model_output"])
    print("模型选择:", eval_item["model_choice"])
    print(f"致信度: {eval_item['confidence']:.4f}")
    print("-" * 50)